In [11]:
import numpy as np
import tensorflow as tf
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_curve, precision_score, f1_score, recall_score

In [2]:
unet = tf.keras.models.load_model(r"C:\Users\aashutosh kumar\Downloads\LD.h5")  # Replace with your model path

In [5]:
intensity = ctrl.Antecedent(np.arange(0, 256, 1), 'intensity')
confidence = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'confidence')
category = ctrl.Consequent(np.arange(0, 2, 0.1), 'category')

intensity['low'] = fuzz.trimf(intensity.universe, [0, 0, 160])
intensity['medium'] = fuzz.trimf(intensity.universe, [80, 160, 240])
intensity['high'] = fuzz.trimf(intensity.universe, [95, 255, 255])

# Define membership functions for confidence
confidence['low'] = fuzz.trimf(confidence.universe, [0, 0, 0.6])
confidence['medium'] = fuzz.trimf(confidence.universe, [0.3, 0.5, 0.7])
confidence['high'] = fuzz.trimf(confidence.universe, [0.4, 1, 1])

# Define membership functions for category
category['background'] = fuzz.trimf(category.universe, [0, 0, 1])
category['lane'] = fuzz.trimf(category.universe, [1, 2, 2])

# Define fuzzy rules
rule1 = ctrl.Rule(intensity['low'] & confidence['low'], category['background'])
rule2 = ctrl.Rule(intensity['high'] & confidence['high'], category['lane'])
rule3 = ctrl.Rule((intensity['medium'] | confidence['medium']), category['lane'])

# Create fuzzy control system
category_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
fuzzy_sim = ctrl.ControlSystemSimulation(category_ctrl)

# Load an input image for prediction (example)
def preprocess_image(image_path):
    """Preprocess the image for the U-Net model."""
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256), color_mode="grayscale")
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    
    # Convert grayscale image to 3 channels
    image = np.repeat(image, 3, axis=-1)  # Repeat the grayscale values across 3 channels
    
    # Expand dimensions to match the input shape (batch_size, height, width, channels)
    image = np.expand_dims(image, axis=0)
    return image


# Replace with your test image
image_path = r"D:\Research\Dataset\train\images\0313-1_2460.jpg"
input_image = preprocess_image(image_path)

# Predict mask using U-Net
predicted_mask = unet.predict(input_image)[0]

# Apply fuzzy logic to refine U-Net predictions
refined_mask = np.zeros_like(predicted_mask)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step


In [6]:
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix

# Function to compute IoU, accuracy, precision, recall, and F1 score
def compute_metrics(pred_mask, ground_truth):
    # Ensure binary masks (0 or 1)
    pred_mask = (pred_mask > 0.5).astype(int)  # Thresholding predicted mask
    ground_truth = (ground_truth > 0.5).astype(int)  # Thresholding ground truth mask

    # Flatten the images to 1D for comparison
    pred_mask_flat = pred_mask.flatten()
    ground_truth_flat = ground_truth.flatten()

    # Confusion matrix (True positives, False positives, True negatives, False negatives)
    tn, fp, fn, tp = confusion_matrix(ground_truth_flat, pred_mask_flat).ravel()

    # Calculate IoU (Intersection over Union)
    iou = tp / (tp + fp + fn)

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Calculate Precision and Recall
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0

    # Calculate F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return iou, accuracy, precision, recall, f1_score

# Function to evaluate U-Net and U-Net + Fuzzy refinement
def evaluate_unet_vs_fuzzy(input_image_path, unet_model, fuzzy_sim, ground_truth_mask):
    input_image = preprocess_image(input_image_path)

    # Predict mask using U-Net
    predicted_mask_unet = unet_model.predict(input_image)[0]

    # Initialize refined mask
    refined_mask_fuzzy = np.zeros_like(predicted_mask_unet)

    # Refine the predicted mask using fuzzy logic
    for i in range(predicted_mask_unet.shape[0]):
        for j in range(predicted_mask_unet.shape[1]):
            # Get pixel intensity and prediction confidence
            intensity_value = input_image[0, i, j, 0] * 255  # Rescale intensity to 0-255
            confidence_value = predicted_mask_unet[i, j, 0]  # Predicted confidence
            
            # Fuzzy inference
            fuzzy_sim.input['intensity'] = intensity_value
            fuzzy_sim.input['confidence'] = confidence_value
            fuzzy_sim.compute()

            # Ensure fuzzy output is available
            if 'category' in fuzzy_sim.output:
                refined_mask_fuzzy[i, j, 0] = fuzzy_sim.output['category']
            else:
                refined_mask_fuzzy[i, j, 0] = 0  # Default value if 'category' is missing

    # Compute metrics for U-Net prediction
    iou_unet, accuracy_unet, precision_unet, recall_unet, f1_score_unet = compute_metrics(predicted_mask_unet, ground_truth_mask)

    # Compute metrics for refined mask (U-Net + Fuzzy)
    iou_fuzzy, accuracy_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy = compute_metrics(refined_mask_fuzzy, ground_truth_mask)

    # Print results for U-Net prediction
    print("Metrics for U-Net Prediction:")
    print(f"IoU: {iou_unet:.4f}")
    print(f"Accuracy: {accuracy_unet:.4f}")
    print(f"Precision: {precision_unet:.4f}")
    print(f"Recall: {recall_unet:.4f}")
    print(f"F1 Score: {f1_score_unet:.4f}")

    # Print results for U-Net + Fuzzy refinement
    print("\nMetrics for U-Net + Fuzzy Refinement:")
    print(f"IoU: {iou_fuzzy:.4f}")
    print(f"Accuracy: {accuracy_fuzzy:.4f}")
    print(f"Precision: {precision_fuzzy:.4f}")
    print(f"Recall: {recall_fuzzy:.4f}")
    print(f"F1 Score: {f1_score_fuzzy:.4f}")

# Example usage
input_image_path = r"D:\Research\Dataset\test\images\0313-1_4580.jpg"  
ground_truth_mask = cv2.imread(r"D:\Research\Dataset\test\masks\0313-1_4580.jpg", cv2.IMREAD_GRAYSCALE)  # Load ground truth mask
ground_truth_mask = cv2.resize(ground_truth_mask, (256, 256))  

evaluate_unet_vs_fuzzy(input_image_path, unet, fuzzy_sim, ground_truth_mask)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
Metrics for U-Net Prediction:
IoU: 0.4521
Accuracy: 0.9649
Precision: 0.8439
Recall: 0.4934
F1 Score: 0.6227

Metrics for U-Net + Fuzzy Refinement:
IoU: 0.0795
Accuracy: 0.3549
Precision: 0.0798
Recall: 0.9498
F1 Score: 0.1473


In [8]:
# Redefine fuzzy membership functions with tighter boundaries
intensity['low'] = fuzz.trapmf(intensity.universe, [0, 0, 80, 150])
intensity['medium'] = fuzz.trapmf(intensity.universe, [80, 150, 200, 230])
intensity['high'] = fuzz.trapmf(intensity.universe, [200, 230, 255, 255])

confidence['low'] = fuzz.trapmf(confidence.universe, [0, 0, 0.3, 0.5])
confidence['medium'] = fuzz.trapmf(confidence.universe, [0.3, 0.5, 0.7, 0.9])
confidence['high'] = fuzz.trapmf(confidence.universe, [0.7, 0.9, 1, 1])

category['background'] = fuzz.trapmf(category.universe, [0, 0, 0.3, 0.5])
category['lane'] = fuzz.trapmf(category.universe, [0.5, 0.7, 1, 1])

# Update rules to reduce false positives
rule1 = ctrl.Rule(intensity['low'] & confidence['low'], category['background'])
rule2 = ctrl.Rule(intensity['high'] & confidence['high'], category['lane'])
rule3 = ctrl.Rule(intensity['medium'] & confidence['medium'], category['lane'])
rule4 = ctrl.Rule(intensity['high'] & confidence['low'], category['background'])

# Create fuzzy control system
category_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
fuzzy_sim = ctrl.ControlSystemSimulation(category_ctrl)

# Modify mask conversion logic
def refine_mask(predicted_mask, input_image, fuzzy_sim):
    refined_mask = np.zeros_like(predicted_mask)
    for i in range(predicted_mask.shape[0]):
        for j in range(predicted_mask.shape[1]):
            intensity_value = input_image[0, i, j, 0] * 255  # Scale intensity to 0-255
            confidence_value = predicted_mask[i, j, 0]

            fuzzy_sim.input['intensity'] = intensity_value
            fuzzy_sim.input['confidence'] = confidence_value
            fuzzy_sim.compute()

            # Apply threshold to fuzzy output to classify into binary categories
            if 'category' in fuzzy_sim.output:
                fuzzy_output = fuzzy_sim.output['category']
                refined_mask[i, j, 0] = 1 if fuzzy_output > 0.5 else 0
            else:
                refined_mask[i, j, 0] = 0

    return refined_mask

# Evaluate the updated system
def evaluate_unet_vs_fuzzy(input_image_path, unet_model, fuzzy_sim, ground_truth_mask):
    input_image = preprocess_image(input_image_path)
    predicted_mask_unet = unet_model.predict(input_image)[0]
    refined_mask_fuzzy = refine_mask(predicted_mask_unet, input_image, fuzzy_sim)

    iou_unet, accuracy_unet, precision_unet, recall_unet, f1_score_unet = compute_metrics(predicted_mask_unet, ground_truth_mask)
    iou_fuzzy, accuracy_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy = compute_metrics(refined_mask_fuzzy, ground_truth_mask)

    print("Metrics for U-Net Prediction:")
    print(f"IoU: {iou_unet:.4f}")
    print(f"Accuracy: {accuracy_unet:.4f}")
    print(f"Precision: {precision_unet:.4f}")
    print(f"Recall: {recall_unet:.4f}")
    print(f"F1 Score: {f1_score_unet:.4f}")

    print("\nMetrics for U-Net + Fuzzy Refinement:")
    print(f"IoU: {iou_fuzzy:.4f}")
    print(f"Accuracy: {accuracy_fuzzy:.4f}")
    print(f"Precision: {precision_fuzzy:.4f}")
    print(f"Recall: {recall_fuzzy:.4f}")
    print(f"F1 Score: {f1_score_fuzzy:.4f}")

# Test the updated system
input_image_path = r"D:\Research\Dataset\test\images\0313-1_4580.jpg"
ground_truth_mask = cv2.imread(r"D:\Research\Dataset\test\masks\0313-1_4580.jpg", cv2.IMREAD_GRAYSCALE)
ground_truth_mask = cv2.resize(ground_truth_mask, (256, 256))

evaluate_unet_vs_fuzzy(input_image_path, unet, fuzzy_sim, ground_truth_mask)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
Metrics for U-Net Prediction:
IoU: 0.4521
Accuracy: 0.9649
Precision: 0.8439
Recall: 0.4934
F1 Score: 0.6227

Metrics for U-Net + Fuzzy Refinement:
IoU: 0.4569
Accuracy: 0.9601
Precision: 0.6943
Recall: 0.5719
F1 Score: 0.6272


In [18]:
import numpy as np
import cv2
import skfuzzy as fuzz
import skfuzzy.control as ctrl

# Refined fuzzy membership functions with tighter boundaries
intensity['low'] = fuzz.trapmf(intensity.universe, [0, 0, 80, 140])
intensity['medium'] = fuzz.trapmf(intensity.universe, [80, 140, 180, 220])
intensity['high'] = fuzz.trapmf(intensity.universe, [180, 220, 255, 255])

confidence['low'] = fuzz.trapmf(confidence.universe, [0, 0, 0.2, 0.4])
confidence['medium'] = fuzz.trapmf(confidence.universe, [0.3, 0.5, 0.7, 0.9])
confidence['high'] = fuzz.trapmf(confidence.universe, [0.6, 0.8, 1, 1])

category['background'] = fuzz.trapmf(category.universe, [0, 0, 0.2, 0.5])
category['lane'] = fuzz.trapmf(category.universe, [0.4, 0.6, 1, 1])

# Enhanced fuzzy rules
rule1 = ctrl.Rule(intensity['low'] & confidence['low'], category['background'])
rule2 = ctrl.Rule(intensity['medium'] & confidence['low'], category['background'])
rule3 = ctrl.Rule(intensity['medium'] & confidence['medium'], category['lane'])
rule4 = ctrl.Rule(intensity['high'] & confidence['high'], category['lane'])
rule5 = ctrl.Rule(intensity['high'] & confidence['low'], category['background'])
rule6 = ctrl.Rule(intensity['low'] & confidence['high'], category['background'])

# Create fuzzy control system with updated rules
category_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
fuzzy_sim = ctrl.ControlSystemSimulation(category_ctrl)

# Update mask conversion logic for refinement
def refine_mask(predicted_mask, input_image, fuzzy_sim):
    refined_mask = np.zeros_like(predicted_mask)
    for i in range(predicted_mask.shape[0]):
        for j in range(predicted_mask.shape[1]):
            intensity_value = input_image[0, i, j, 0] * 255  # Scale intensity to 0-255
            confidence_value = predicted_mask[i, j, 0]

            fuzzy_sim.input['intensity'] = intensity_value
            fuzzy_sim.input['confidence'] = confidence_value
            fuzzy_sim.compute()

            if 'category' in fuzzy_sim.output:
                fuzzy_output = fuzzy_sim.output['category']
                refined_mask[i, j, 0] = 1 if fuzzy_output > 0.5 else 0
            else:
                refined_mask[i, j, 0] = 0

    return refined_mask

# Evaluate the updated system
def evaluate_unet_vs_fuzzy(input_image_path, unet_model, fuzzy_sim, ground_truth_mask):
    input_image = preprocess_image(input_image_path)
    predicted_mask_unet = unet_model.predict(input_image)[0]
    refined_mask_fuzzy = refine_mask(predicted_mask_unet, input_image, fuzzy_sim)

    iou_unet, accuracy_unet, precision_unet, recall_unet, f1_score_unet = compute_metrics(predicted_mask_unet, ground_truth_mask)
    iou_fuzzy, accuracy_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy = compute_metrics(refined_mask_fuzzy, ground_truth_mask)

    print("Metrics for U-Net Prediction:")
    print(f"IoU: {iou_unet:.4f}")
    print(f"Accuracy: {accuracy_unet:.4f}")
    print(f"Precision: {precision_unet:.4f}")
    print(f"Recall: {recall_unet:.4f}")
    print(f"F1 Score: {f1_score_unet:.4f}")

    print("\nMetrics for U-Net + Fuzzy Refinement:")
    print(f"IoU: {iou_fuzzy:.4f}")
    print(f"Accuracy: {accuracy_fuzzy:.4f}")
    print(f"Precision: {precision_fuzzy:.4f}")
    print(f"Recall: {recall_fuzzy:.4f}")
    print(f"F1 Score: {f1_score_fuzzy:.4f}")

# Test the updated system
input_image_path = r"D:\Research\Dataset\test\images\0313-1_4580.jpg"
ground_truth_mask = cv2.imread(r"D:\Research\Dataset\test\masks\0313-1_4580.jpg", cv2.IMREAD_GRAYSCALE)
ground_truth_mask = cv2.resize(ground_truth_mask, (256, 256))

evaluate_unet_vs_fuzzy(input_image_path, unet, fuzzy_sim, ground_truth_mask)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets